In [3]:
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import tensorflow as tf


class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    # Extract frozen graph from tar archive.
    tar_file = tarfile.open(tarball_path)
    for tar_info in tar_file.getmembers():
      if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.GraphDef.FromString(file_handle.read())
        break

    tar_file.close()

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map

In [4]:
download_path = '/home/stmoon/Test/models/deeplabv3_cityscapes_train_2018_02_06.tar.gz'
MODEL = DeepLabModel(download_path)

In [9]:
img_path = '/home/stmoon/Test/PSPNet-tensorflow/input/frame0374.png'
img = Image.open(img_path) 

resized_im, seg_map = MODEL.run(img)
p = resized_im.load()

for i in range(resized_im.size[0]) :
    for j in range(resized_im.size[1]) :
        if seg_map[j,i] == 0 :
            p[i,j] = tuple([int(x*0.3) for x in p[i,j]])
            
resized_im.save("./test.png")


In [ ]:
import glob

dir_path = '/home/stmoon/Project/AE590/data/out3/'
out_path = '/home/stmoon/Project/AE590/deeplab/out/out3'
file_list = sorted(glob.glob(dir_path + '/*.png'))

for f in file_list :
    img = Image.open(f) 

    resized_im, seg_map = MODEL.run(img)
    p = resized_im.load()

    for i in range(resized_im.size[0]) :
        for j in range(resized_im.size[1]) :
            if seg_map[j,i] == 0 :
                p[i,j] = tuple([int(x*0.3) for x in p[i,j]])

    print(out_path + '/' + f.split('/')[-1])        
    resized_im.save(out_path + '/' + f.split('/')[-1])
    

/home/stmoon/Project/AE590/deeplab/out/out3/image_0000_-0.07.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0001_-0.08.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0002_-0.08.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0003_-0.09.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0004_-0.09.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0005_-0.08.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0006_-0.09.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0007_-0.08.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0008_-0.09.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0009_-0.09.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0010_-0.10.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0011_-0.09.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0012_-0.08.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0013_-0.09.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0014_-0.10.png
/home/stmoon/Project/AE59

/home/stmoon/Project/AE590/deeplab/out/out3/image_0127_25.84.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0128_25.85.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0129_25.83.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0130_25.84.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0131_25.85.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0132_25.83.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0133_25.84.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0134_25.85.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0135_25.85.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0136_25.85.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0137_25.84.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0138_25.86.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0139_25.86.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0140_25.87.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0141_25.86.png
/home/stmoon/Project/AE59

/home/stmoon/Project/AE590/deeplab/out/out3/image_0254_-0.34.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0255_-0.33.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0256_-0.34.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0257_-0.32.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0258_-0.33.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0259_-0.32.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0260_-0.32.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0261_-0.31.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0262_-0.32.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0263_-0.30.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0264_-0.32.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0265_-0.31.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0266_-0.31.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0267_-0.31.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0268_-0.32.png
/home/stmoon/Project/AE59

/home/stmoon/Project/AE590/deeplab/out/out3/image_0381_2.84.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0382_2.91.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0383_2.97.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0384_3.09.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0385_3.18.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0386_3.37.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0387_3.43.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0388_3.49.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0389_3.56.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0390_3.62.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0391_3.80.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0392_3.99.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0393_4.09.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0394_4.26.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0395_4.41.png
/home/stmoon/Project/AE590/deeplab/out/o

/home/stmoon/Project/AE590/deeplab/out/out3/image_0510_3.98.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0511_3.97.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0512_3.96.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0513_3.94.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0514_3.97.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0515_3.98.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0516_3.98.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0517_3.98.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0518_3.99.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0519_3.98.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0520_3.98.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0521_4.00.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0522_3.99.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0523_3.99.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0524_4.00.png
/home/stmoon/Project/AE590/deeplab/out/o

/home/stmoon/Project/AE590/deeplab/out/out3/image_0639_6.13.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0640_6.04.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0641_5.95.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0642_5.70.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0643_5.61.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0644_5.50.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0645_5.40.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0646_5.30.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0647_5.20.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0648_5.11.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0649_4.87.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0650_4.78.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0651_4.43.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0652_4.33.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0653_4.24.png
/home/stmoon/Project/AE590/deeplab/out/o

/home/stmoon/Project/AE590/deeplab/out/out3/image_0767_25.29.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0768_25.29.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0769_25.30.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0770_25.30.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0771_25.31.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0772_25.32.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0773_25.33.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0774_25.32.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0775_25.34.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0776_25.35.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0777_25.35.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0778_25.35.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0779_25.35.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0780_25.34.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0781_25.33.png
/home/stmoon/Project/AE59

/home/stmoon/Project/AE590/deeplab/out/out3/image_0894_11.04.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0895_11.01.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0896_10.98.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0897_10.95.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0898_10.91.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0899_10.90.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0900_10.90.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0901_10.91.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0902_10.91.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0903_10.89.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0904_10.86.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0905_10.88.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0906_10.87.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0907_10.89.png
/home/stmoon/Project/AE590/deeplab/out/out3/image_0908_10.89.png
/home/stmoon/Project/AE59